Este codigo se aplica solo a titulos en español 

In [28]:
import pandas as pd 
ruta_csv = 'C:/Users/Allis/OneDrive/UniversidadCenfo2025/Practicaclase2procesos/STEM_LADAINA/7_Datasets_Variaciones/Dataset_DivididoPorIdioma.csv'
df = pd.read_csv(ruta_csv)
print(df.head())

         Fuente            Categoría  \
0  Computrabajo  Software Developers   
1  Computrabajo  Software Developers   
2  Computrabajo  Software Developers   
3  Computrabajo       Web Developers   
4  Computrabajo       Web Developers   

                                                       Título  \
0                                      Analista desarrollador   
1                                   Programador Sistemas .Net   
2  Técnico en Soporte Informático San José Cartago y Alajuela   
3                            Desarrollador Senior TI (Oracle)   
4                              Técnico en Soporte informático   

                                                            Empresa  \
0  FACTOR K QUINIENTOS SEIS SC SOCIEDAD DE RESPONSABILIDAD LIMITADA   
1                                    Architect Technology Solutions   
2                                                 SOPORTEXPERTO.COM   
3                                                           Asembis   
4            

In [51]:
import re
import pandas as pd
import os
from collections import Counter

# === CARGAR DATASET ===


# ========================================
# 1️⃣ DICCIONARIO DE SENIORITY / NIVEL
# ========================================
seniority_dict = {
    # Junior
    "jr": "junior", "jr.": "junior", "junior": "junior", "joven": "junior",
    "principiante": "junior", "trainee": "junior", "entry": "junior", 
    "nivel inicial": "junior", "inicial": "junior", "asociado": "junior",
    "auxiliar": "junior", "asistente": "junior", "ayudante": "junior",
    "aprendiz": "junior", "becario": "junior", "estudiante": "junior",
    "practicante": "junior", "intern": "junior", "pasante": "junior",
    
    # Semi-Senior
    "ssr": "semisenior", "semi senior": "semisenior", "semi-senior": "semisenior",
    "intermedio": "semisenior", "intermedia": "semisenior", "mid": "semisenior",
    "mid level": "semisenior", "pleno": "semisenior", "semi": "semisenior",
    "semi avanzado": "semisenior", "experiencia media": "semisenior",
    
    # Senior
    "sr": "senior", "sr.": "senior", "senior": "senior", "avanzado": "senior",
    "experto": "senior", "especialista": "senior", "especializado": "senior",
    "master": "senior", "advanced": "senior", "high level": "senior",
    "nivel alto": "senior", "alto nivel": "senior", "experimentado": "senior",
    "calificado": "senior", "certificado": "senior", "profesional": "senior",
    
    # Liderazgo
    "lead": "senior", "líder": "senior", "lider": "senior", "principal": "senior",
    "chief": "senior", "head": "senior", "manager": "senior", "mánager": "senior",
    "coordinador": "senior", "coordinadora": "senior", "supervisor": "senior",
    "supervisora": "senior", "jefe": "senior", "jefa": "senior", "gerente": "senior",
    "director": "senior", "directora": "senior", "regente": "senior",
    "encargado": "senior", "encargada": "senior", "responsable": "senior",
    "capitan": "senior", "capitán": "senior", "gestor": "senior", "gestora": "senior",
    "subgerente": "senior", "ejecutivo": "senior", "ejecutiva": "senior",
    "consultor": "senior", "consultora": "senior", "arquitecto": "senior",
    "arquitecta": "senior", "strategist": "senior", "analyst": "senior",
    "consultant": "senior", "specialist": "senior", "expert": "senior"
}

# ========================================
# 2️⃣ DICCIONARIO DE MODALIDAD / JORNADA
# ========================================
modalidad_dict = {
    # Remoto
    "remoto": "remoto", "remote": "remoto", "teletrabajo": "remoto", 
    "casa": "remoto", "home": "remoto", "home office": "remoto",
    "teletrabajo": "remoto", "virtual": "remoto", "online": "remoto",
    "distancia": "remoto", "distante": "remoto", "work from home": "remoto",
    "wfh": "remoto", "telecommuting": "remoto", "telework": "remoto",
    
    # Híbrido
    "híbrido": "hibrido", "hibrido": "hibrido", "hybrid": "hibrido",
    "mixto": "hibrido", "combinado": "hibrido", "flexible": "hibrido",
    "semipresencial": "hibrido", "semi presencial": "hibrido",
    
    # Presencial
    "presencial": "presencial", "onsite": "presencial", "oficina": "presencial",
    "local": "presencial", "in situ": "presencial", "en sitio": "presencial",
    "fisico": "presencial", "físico": "presencial", "on site": "presencial",
    "office": "presencial", "location": "presencial", "en empresa": "presencial",
    
    # Tiempo completo
    "full time": "tiempo completo", "jornada completa": "tiempo completo", 
    "tiempo completo": "tiempo completo", "completo": "tiempo completo",
    "full-time": "tiempo completo", "fulltime": "tiempo completo",
    "jornada completa": "tiempo completo", "completa": "tiempo completo",
    "40 horas": "tiempo completo", "cuarenta horas": "tiempo completo",
    "jornada entera": "tiempo completo", "entera": "tiempo completo",
    
    # Medio tiempo
    "part time": "medio tiempo", "medio tiempo": "medio tiempo", 
    "tiempo parcial": "medio tiempo", "parcial": "medio tiempo",
    "part-time": "medio tiempo", "parttime": "medio tiempo",
    "media jornada": "medio tiempo", "mitad de tiempo": "medio tiempo",
    "20 horas": "medio tiempo", "veinte horas": "medio tiempo",
    "media jornada": "medio tiempo", "jornada parcial": "medio tiempo",
    
    # Horarios específicos
    "jornada diurna": "diurna", "diurno": "diurna", "dia": "diurna", "día": "diurna",
    "jornada nocturna": "nocturna", "nocturno": "nocturna", "noche": "nocturna",
    "jornada matutina": "diurna", "matutino": "diurna", "mañana": "diurna",
    "jornada vespertina": "diurna", "vespertino": "diurna", "tarde": "diurna",
    "rotativo": "rotativo", "rotativa": "rotativo", "turnos": "rotativo",
    "por turnos": "rotativo", "turno rotativo": "rotativo",
    
    # Temporalidad
    "temporal": "temporal", "por horas": "temporal", "contrato": "temporal",
    "definido": "temporal", "temporario": "temporal", "proyecto": "temporal",
    "duración determinada": "temporal", "determinado": "temporal",
    "seasonal": "temporal", "estacional": "temporal", "interino": "temporal",
    "suplente": "temporal", "reemplazo": "temporal", "substituto": "temporal",
    "freelance": "freelance", "freelancer": "freelance", "independiente": "freelance",
    "consultoría": "freelance", "por proyecto": "freelance", "por honorarios": "freelance",
    
    # Prácticas
    "practicante": "intern", "internship": "intern", "pasante": "intern",
    "practicas": "intern", "prácticas": "intern", "trainnee": "intern",
    "aprendiz": "intern", "becario": "intern", "residente": "intern",
    
    # Junior
    "jr": "junior", "jr.": "junior", "principiante": "junior", "ayudante": "junior",
    "inicial": "junior", "nivel inicial": "junior", "basico": "junior", "básico": "junior",
    
    # Semi-Senior  
    "semi": "semisenior", "semi-avanzado": "semisenior", "intermedia": "semisenior",
    "medio": "semisenior", "pleno": "semisenior",
    
    # Senior
    "experimentado": "senior", "calificado": "senior", "especializado": "senior",
    "avanzado": "senior", "experto": "senior", "master": "senior",
    
    # Liderazgo
    "supervisora": "senior", "jefa": "senior", "directora": "senior", "gerenta": "senior",
    "coordinadora": "senior", "encargada": "senior", "responsable": "senior",
    "gestora": "senior", "ejecutiva": "senior", "consultora": "senior",
    "arquitecta": "senior", "regente": "senior", "subgerente": "senior"  
}
# ========================================
# 3️ DICCIONARIO DE SINÓNIMOS DE PUESTOS
# ========================================

puesto_dict = {
    # ========== DESARROLLO DE SOFTWARE ==========
    "desarrollador": "developer", "programador": "developer", "developer": "developer",
    "dev": "developer", "codificador": "developer", "coder": "developer",
    "software": "software", "sistemas": "sistemas", "informatica": "informatica",
    "computacion": "computacion", "computación": "computacion",
    
    # Backend
    "backend": "backend", "back-end": "backend", "back end": "backend",
    "servidor": "backend", "server": "backend", "server side": "backend",
    "lado del servidor": "backend",
    
    # Frontend
    "frontend": "frontend", "front-end": "frontend", "front end": "frontend",
    "cliente": "frontend", "client": "frontend", "client side": "frontend",
    "lado del cliente": "frontend", "interfaz": "frontend", "ui": "frontend",
    
    # Fullstack
    "fullstack": "fullstack", "full-stack": "fullstack", "full stack": "fullstack",
    "completo": "fullstack", "integral": "fullstack",
    
    # Mobile
    "mobile": "mobile", "móvil": "mobile", "android": "mobile", "ios": "mobile",
    "aplicaciones moviles": "mobile", "apps": "mobile", "aplicaciones": "mobile",
    
    # Especialidades desarrollo
    "web": "web", "aplicaciones web": "web", "sitios web": "web",
    "software engineer": "ingeniero software", "ingeniero de software": "ingeniero software",
    "software developer": "developer", "application developer": "developer",
    "devops": "devops", "development operations": "devops",
    "qa": "qa", "quality assurance": "qa", "control calidad": "qa",
    "tester": "tester", "testing": "tester", "pruebas": "tester",
    "arquitecto": "arquitecto", "arquitecta": "arquitecto", "architect": "arquitecto",
    "analista programador": "developer", "analista desarrollador": "developer",
    "diseñador software": "developer", "diseñadora software": "developer",
    
    # ========== DATOS E INTELIGENCIA ARTIFICIAL ==========
    "data": "datos", "datos": "datos", "data engineer": "ingeniero datos",
    "ingeniero de datos": "ingeniero datos", "data scientist": "data scientist",
    "científico de datos": "data scientist", "cientifico datos": "data scientist",
    "ml": "machine learning", "machine learning": "machine learning",
    "aprendizaje automático": "machine learning", "aprendizaje automatico": "machine learning",
    "ai": "ia", "artificial intelligence": "ia", "inteligencia artificial": "ia",
    "analista": "analista", "analista de datos": "analista datos",
    "data analyst": "analista datos", "business intelligence": "business intelligence",
    "bi": "business intelligence", "inteligencia negocio": "business intelligence",
    "data analytics": "analista datos", "analítica": "analista datos",
    "estadistico": "estadistico", "estadística": "estadistico", "statistician": "estadistico",
    "investigador": "investigador", "investigadora": "investigador", "research": "investigador",
    
    # ========== INFRAESTRUCTURA Y REDES ==========
    "infraestructura": "infraestructura", "infrastructure": "infraestructura",
    "redes": "redes", "network": "redes", "networking": "redes",
    "sysadmin": "administrador sistemas", "system administrator": "administrador sistemas",
    "administrador": "administrador", "administradora": "administrador",
    "administrador de sistemas": "administrador sistemas", "admin": "administrador",
    "cloud": "cloud", "nube": "cloud", "cloud computing": "cloud",
    "devops": "devops", "sre": "sre", "site reliability": "sre",
    "seguridad": "seguridad", "security": "seguridad", "ciberseguridad": "seguridad",
    "cybersecurity": "seguridad", "hacking": "seguridad", "hacker": "seguridad",
    
    # ========== SOPORTE TÉCNICO ==========
    "soporte": "soporte", "soporte técnico": "soporte", "technical support": "soporte",
    "soporte informático": "soporte", "soporte tecnico": "soporte",
    "helpdesk": "soporte", "help desk": "soporte", "service desk": "soporte",
    "mesa de ayuda": "soporte", "asistencia técnica": "soporte",
    "soporte usuario": "soporte", "user support": "soporte",
    "soporte cliente": "soporte", "customer support": "soporte",
    
    # ========== INGENIERÍAS ==========
    "ingeniero": "ingeniero", "ingeniera": "ingeniero", "ing": "ingeniero",
    "engineer": "ingeniero", "engineering": "ingeniero",
    
    # Civil
    "civil": "ingeniero civil", "ingeniero civil": "ingeniero civil",
    "civil engineer": "ingeniero civil", "construcción": "ingeniero civil",
    "construccion": "ingeniero civil", "construction": "ingeniero civil",
    "estructuras": "ingeniero civil", "structural": "ingeniero civil",
    
    # Eléctrico
    "eléctrico": "ingeniero electrico", "electrico": "ingeniero electrico",
    "ingeniero eléctrico": "ingeniero electrico", "electrical": "ingeniero electrico",
    "electricidad": "ingeniero electrico", "electricista": "ingeniero electrico",
    
    # Electromecánico
    "electromecánico": "ingeniero electromecanico", "electromecanico": "ingeniero electromecanico",
    "ingeniero electromecánico": "ingeniero electromecanico", "electromechanical": "ingeniero electromecanico",
    "electromecanico": "electronico", "electrotecnia": "electromecanico",
    
    # Industrial
    "industrial": "ingeniero industrial", "ingeniero industrial": "ingeniero industrial",
    "industrial engineer": "ingeniero industrial", "producción": "ingeniero industrial",
    "produccion": "ingeniero industrial", "manufactura": "ingeniero industrial",
    
    # Mecánico
    "mecánico": "ingeniero mecanico", "mecanico": "ingeniero mecanico",
    "ingeniero mecánico": "ingeniero mecanico", "mechanical": "ingeniero mecanico",
    "mecanica": "ingeniero mecanico", "mecánica": "ingeniero mecanico",
    
    # Otras ingenierías
    "calidad": "ingeniero calidad", "ingeniero de calidad": "ingeniero calidad",
    "quality": "ingeniero calidad", "proyectos": "ingeniero proyectos",
    "ingeniero de proyectos": "ingeniero proyectos", "project engineer": "ingeniero proyectos",
    "proceso": "ingeniero procesos", "ingeniero de proceso": "ingeniero procesos",
    "process engineer": "ingeniero procesos", "manufactura": "ingeniero manufactura",
    "manufacturing": "ingeniero manufactura", "produccion": "ingeniero produccion",
    "production": "ingeniero produccion", "mantenimiento": "ingeniero mantenimiento",
    "maintenance": "ingeniero mantenimiento", "residente": "ingeniero residente",
    "ingeniero residente": "ingeniero residente", "controlador": "ingeniero controlador",
    "ingeniero controlador": "ingeniero controlador", "agronomo": "ingeniero agronomo",
    "ingeniero agronomo": "ingeniero agronomo", "agrónomo": "ingeniero agronomo",
    
    # ========== TÉCNICOS ==========
    "técnico": "tecnico", "tecnico": "tecnico", "technician": "tecnico",
    "tec": "tecnico", "tecnologo": "tecnico", "tecnóloga": "tecnico",
    "tecnólogo": "tecnico", "tecnica": "tecnico", "técnica": "tecnico",
    
    # Especialidades técnicas
    "electromecánico": "tecnico electromecanico", "electromecanico": "tecnico electromecanico",
    "técnico electromecánico": "tecnico electromecanico", "eléctrico": "tecnico electrico",
    "electrico": "tecnico electrico", "técnico eléctrico": "tecnico electrico",
    "electricista": "tecnico electrico", "soporte": "tecnico soporte",
    "técnico en soporte": "tecnico soporte", "técnico de soporte": "tecnico soporte",
    "mantenimiento": "tecnico mantenimiento", "técnico en mantenimiento": "tecnico mantenimiento",
    "técnico mantenimiento": "tecnico mantenimiento", "refrigeración": "tecnico refrigeracion",
    "refrigeracion": "tecnico refrigeracion", "técnico en refrigeración": "tecnico refrigeracion",
    "técnico refrigeracion": "tecnico refrigeracion", "imágenes": "tecnico imagenes",
    "imagenes": "tecnico imagenes", "técnico en imágenes": "tecnico imagenes",
    "técnico imágenes": "tecnico imagenes", "agricola": "tecnico agricola",
    "agrícola": "tecnico agricola", "técnico agrícola": "tecnico agricola",
    "técnico agricola": "tecnico agricola", "mecánico": "tecnico mecanico",
    "mecanico": "tecnico mecanico", "técnico mecánico": "tecnico mecanico",
    "técnico mecanico": "tecnico mecanico", "electronico": "tecnico electronico",
    "electrónico": "tecnico electronico", "técnico electrónico": "tecnico electronico",
    "técnico electronico": "tecnico electronico", "informático": "tecnico informatico",
    "informatico": "tecnico informatico", "técnico informático": "tecnico informatico",
    "técnico informatico": "tecnico informatico", "computación": "tecnico computacion",
    "computacion": "tecnico computacion", "técnico en computación": "tecnico computacion",
    "técnico computacion": "tecnico computacion",
    
    # ========== DISEÑO ==========
    "diseñador": "diseñador", "diseñadora": "diseñador", "designer": "diseñador",
    "design": "diseñador", "diseño": "diseñador", "ux": "diseñador ux",
    "user experience": "diseñador ux", "experiencia usuario": "diseñador ux",
    "ui": "diseñador ui", "user interface": "diseñador ui", "interfaz usuario": "diseñador ui",
    "ux/ui": "diseñador uxui", "ux ui": "diseñador uxui", "gráfico": "diseñador grafico",
    "grafico": "diseñador grafico", "diseñador gráfico": "diseñador grafico",
    "diseñador grafico": "diseñador grafico", "graphic": "diseñador grafico",
    "publicitario": "diseñador publicitario", "diseñador publicitario": "diseñador publicitario",
    "advertising": "diseñador publicitario", "multimedia": "diseñador multimedia",
    "diseñador multimedia": "diseñador multimedia", "web": "diseñador web",
    "diseñador web": "diseñador web", "web designer": "diseñador web",
    
    # ========== GESTIÓN Y LIDERAZGO ==========
    "gerente": "gerente", "manager": "gerente", "mánager": "gerente",
    "director": "director", "directora": "director", "director": "director",
    "jefe": "jefe", "jefa": "jefe", "chief": "jefe", "head": "jefe",
    "coordinador": "coordinador", "coordinadora": "coordinador", "coordinator": "coordinador",
    "supervisor": "supervisor", "supervisora": "supervisor", "supervisor": "supervisor",
    "líder": "lider", "lider": "lider", "lead": "lider", "leader": "lider",
    "project manager": "gerente proyecto", "gestor": "gestor", "gestora": "gestor",
    "encargado": "encargado", "encargada": "encargado", "responsable": "responsable",
    "regente": "regente", "subgerente": "subgerente", "ejecutivo": "ejecutivo",
    "ejecutiva": "ejecutivo", "executive": "ejecutivo",
    
    # ========== COMERCIAL/VENTAS ==========
    "vendedor": "vendedor", "vendedora": "vendedor", "sales": "vendedor",
    "seller": "vendedor", "comercial": "vendedor", "commercial": "vendedor",
    "ventas": "vendedor", "salesperson": "vendedor", "salesman": "vendedor",
    "saleswoman": "vendedor", "ejecutivo": "ejecutivo", "ejecutiva": "ejecutivo",
    "asesor": "asesor", "asesora": "asesor", "advisor": "asesor", "consultant": "asesor",
    "vendedor tecnico": "vendedor tecnico", "technical sales": "vendedor tecnico",
    "desarrollador comercial": "vendedor", "ejecutivo comercial": "vendedor",
    "account": "vendedor", "account executive": "vendedor", "account manager": "vendedor",
    
    # ========== PROFESIONES ESPECIALIZADAS ==========
    "farmacéutico": "farmaceutico", "farmaceutico": "farmaceutico", "pharmacist": "farmaceutico",
    "microbiólogo": "microbiologo", "microbiologo": "microbiologo", "microbiologist": "microbiologo",
    "agrónomo": "agronomo", "agronomo": "agronomo", "agronomist": "agronomo",
    "veterinario": "veterinario", "veterinaria": "veterinario", "veterinary": "veterinario",
    "electromedicina": "electromedicina", "electromedico": "electromedicina",
    "radiologo": "radiologo", "radiología": "radiologo", "radiology": "radiologo",
    "imagenes medicas": "imagenes medicas", "medical imaging": "imagenes medicas",
    "quimico": "quimico", "químico": "quimico", "chemist": "quimico",
    "fisico": "fisico", "físico": "fisico", "physicist": "fisico",
    "biologo": "biologo", "biólogo": "biologo", "biologist": "biologo",
    "medico": "medico", "médico": "medico", "doctor": "medico", "physician": "medico",
    "enfermero": "enfermero", "enfermera": "enfermero", "nurse": "enfermero",
    "psiquiatra": "psiquiatra", "psychiatrist": "psiquiatra",
    
    # ========== FINANZAS Y CONTABILIDAD ==========
    "contador": "contador", "contadora": "contador", "accountant": "contador",
    "finanzas": "finanzas", "finance": "finanzas", "financiero": "finanzas",
    "economista": "economista", "economist": "economista",
    "auditor": "auditor", "auditora": "auditor", "auditor": "auditor",
    "tesorero": "tesorero", "tesorera": "tesorero", "treasurer": "tesorero",
    "analista financiero": "analista financiero", "financial analyst": "analista financiero",
    
    # ========== RECURSOS HUMANOS ==========
    "recursos humanos": "recursos humanos", "human resources": "recursos humanos",
    "rh": "recursos humanos", "hr": "recursos humanos", "talento humano": "recursos humanos",
    "reclutador": "reclutador", "reclutadora": "reclutador", "recruiter": "reclutador",
    "selección": "reclutador", "seleccion": "reclutador", "selection": "reclutador",
    
    # ========== MARKETING Y COMUNICACIÓN ==========
    "marketing": "marketing", "mercadeo": "marketing", "mercadotecnia": "marketing",
    "comunicación": "comunicacion", "comunicacion": "comunicacion", "communication": "comunicacion",
    "publicidad": "publicidad", "advertising": "publicidad", "advertizing": "publicidad",
    "redes sociales": "redes sociales", "social media": "redes sociales", "social": "redes sociales",
    "community manager": "redes sociales", "cm": "redes sociales",
    "content": "contenido", "contenido": "contenido", "creador": "contenido",
    "creadora": "contenido", "creator": "contenido", "copywriter": "contenido",
    
    # ========== OPERACIONES Y LOGÍSTICA ==========
    "logistica": "logistica", "logística": "logistica", "logistics": "logistica",
    "operaciones": "operaciones", "operations": "operaciones",
    "cadena suministro": "logistica", "supply chain": "logistica",
    "almacen": "almacen", "almacén": "almacen", "warehouse": "almacen",
    "inventario": "inventario", "inventory": "inventario",
    "distribucion": "distribucion", "distribución": "distribucion", "distribution": "distribucion",
    "compras": "compras", "purchasing": "compras", "procurement": "compras",
    
    # ========== ATENCIÓN AL CLIENTE ==========
    "atencion al cliente": "atencion cliente", "atención al cliente": "atencion cliente",
    "customer service": "atencion cliente", "servicio al cliente": "atencion cliente",
    "call center": "atencion cliente", "contact center": "atencion cliente",
    "representante": "representante", "representative": "representante",
    "agente": "agente", "agent": "agente", "operador": "operador", "operator": "operador",
    "serv_cliente": "Servicio al Cliente",
    "gest_operativa": "Gestión Operativa",
    
    # ========== ADMINISTRATIVO ==========
    "administrativo": "administrativo", "administrativa": "administrativo",
    "administrative": "administrativo", "asistente": "asistente", "assistant": "asistente",
    "secretario": "secretario", "secretaria": "secretario", "secretary": "secretario",
    "recepcionista": "recepcionista", "receptionist": "recepcionista",
    "auxiliar": "auxiliar", "auxiliary": "auxiliar", "asistente": "asistente",
    
    # ========== LEGAL ==========
    "abogado": "abogado", "abogada": "abogado", "lawyer": "abogado",
    "legal": "legal", "asesor legal": "legal", "legal advisor": "legal",
    "notario": "notario", "notaria": "notario", "notary": "notario",
    
    # ========== EDUCACIÓN ==========
    "profesor": "profesor", "profesora": "profesor", "teacher": "profesor",
    "docente": "profesor", "educator": "profesor", "maestro": "profesor",
    "maestra": "profesor", "instructor": "instructor", "instructora": "instructor",
    "trainer": "instructor", "facilitador": "instructor", "facilitadora": "instructor",
    
    # ========== SEGURIDAD ==========
    "seguridad": "seguridad", "security": "seguridad", "guardia": "seguridad",
    "vigilante": "seguridad", "watchman": "seguridad", "oficial": "seguridad",
    "officer": "seguridad", "protector": "seguridad", "protection": "seguridad",
    
    # ========== CONSTRUCCIÓN Y OFICIOS ==========
    "constructor": "constructor", "constructora": "constructor", "builder": "constructor",
    "carpintero": "carpintero", "carpintera": "carpintero", "carpenter": "carpintero",
    "plomero": "plomero", "plomera": "plomero", "plumber": "plomero",
    "electricista": "electricista", "electrician": "electricista",
    "pintor": "pintor", "pintora": "pintor", "painter": "pintor",
    "albañil": "albañil", "mason": "albañil", "bricklayer": "albañil",
    
    # ========== SERVICIOS ==========
    "chef": "chef", "cocinero": "chef", "cocinera": "chef", "cook": "chef",
    "mesero": "mesero", "mesera": "mesero", "waiter": "mesero", "waitress": "mesero",
    "bartender": "bartender", "barman": "bartender", "barmaid": "bartender",
    "conserje": "conserje", "janitor": "conserje", "custodian": "conserje",
    "limpieza": "limpieza", "cleaner": "limpieza", "cleaning": "limpieza",
    
    # ========== OTROS ==========
    "consultor": "consultor", "consultora": "consultor", "consultant": "consultor",
    "analista": "analista", "analyst": "analista", "especialista": "especialista",
    "specialist": "especialista", "coach": "coach", "entrenador": "coach",
    "entrenadora": "coach", "terapeuta": "terapeuta", "therapist": "terapeuta",
    "investigador": "investigador", "investigadora": "investigador", "researcher": "investigador",
    "científico": "cientifico", "científica": "cientifico", "scientist": "cientifico",
    
        # Nuevas especialidades técnicas
    "electrotecnia": "tecnico electrico", "electrotecnico": "tecnico electrico",
    "maquinaria": "tecnico maquinaria", "maquinaria pesada": "tecnico maquinaria",
    "refrigeracion": "tecnico refrigeracion", "refrigeración": "tecnico refrigeracion",
    "climatizacion": "tecnico refrigeracion", "climatización": "tecnico refrigeracion",
    "paletizado": "operador", "paletizador": "operador",
    
    # Especialidades médicas y científicas
    "electromedicina": "tecnico electromedicina", "electromédico": "tecnico electromedicina",
    "imagenes medicas": "tecnico imagenes", "imágenes médicas": "tecnico imagenes",
    "diagnosticas": "tecnico imagenes", "diagnósticas": "tecnico imagenes",
    "radiologo": "tecnico radiologia", "radiología": "tecnico radiologia",
    
    # Agricultura y veterinaria
    "agronomo": "ingeniero agronomo", "agrónomo": "ingeniero agronomo",
    "agricola": "tecnico agricola", "agrícola": "tecnico agricola",
    "veterinario": "veterinario", "veterinaria": "veterinario",
    
    # Seguridad y protección
    "seguridad": "seguridad", "vigilante": "seguridad", "guardia": "seguridad",
    "oficial": "seguridad", "protector": "seguridad", "monitoreo": "seguridad",
    
    # Operaciones y producción
    "operador": "operador", "operario": "operador", "operativa": "operador",
    "produccion": "operador", "producción": "operador", "empaque": "operador",
    "empacadora": "operador", "etiquetadora": "operador",
    
    # Comercial y ventas específicas
    "preventa": "vendedor tecnico", "pre-venta": "vendedor tecnico",
    "desarrollador comercial": "vendedor", "ejecutivo comercial": "vendedor",
    
    # Finanzas y administración
    "credito": "analista credito", "crédito": "analista credito",
    "aprobacion": "analista credito", "aprobación": "analista credito",
    
    # Nuevos roles tecnológicos
    "aprovisionamiento": "tecnico redes", "provisioning": "tecnico redes",
    "fibra optica": "tecnico redes", "fibra óptica": "tecnico redes",
    "averias": "tecnico soporte", "averías": "tecnico soporte",
    
    # Especialidades de diseño
    "publicitario": "diseñador publicitario", "publicidad": "diseñador publicitario",
    "diseno": "diseñador", "diseño": "diseñador",
    
    # Gestión y administración
    "licitaciones": "administrativo", "licitacion": "administrativo",
    "administracion": "administrativo", "administración": "administrativo",
   #categorias extra 
    "electromedicina": "tecnico electromedicina",
    "imagenes diagnosticas": "tecnico imagenes medicas", 
    "refrigeracion": "tecnico refrigeracion",
    "maquinaria pesada": "tecnico maquinaria",
    "fibra optica": "tecnico telecomunicaciones",
    "seguridad alimentaria": "auditor calidad",
    "pet sitter": "cuidador mascotas",
    
    
    "refrigeracion": "tecnico refrigeracion",
    "refrigeración": "tecnico refrigeracion", 
    "climatizacion": "tecnico refrigeracion",
    "climatización": "tecnico refrigeracion",
    
    "maquinaria pesada": "tecnico maquinaria",
    "maquinaria": "tecnico maquinaria",
    
    "fibra optica": "tecnico telecomunicaciones",
    "fibra óptica": "tecnico telecomunicaciones",
    "telecomunicaciones": "tecnico telecomunicaciones",
    
    "seguridad alimentaria": "auditor calidad",
    "inocuidad": "auditor calidad",
    "auditor alimentario": "auditor calidad",
    
    "pet sitter": "cuidador mascotas",
    "cuidador": "cuidador mascotas", 
    "cuidadora": "cuidador mascotas",
    
    # Especialidades de construcción
    "neumatica": "tecnico neumatica", 
    "neumática": "tecnico neumatica",
    "pneumatica": "tecnico neumatica",
    
    # Especialidades agrícolas
    "buenas practicas": "tecnico agricola",
    "buenas prácticas": "tecnico agricola",
    "bpa": "tecnico agricola",
    
    # Especialidades de soporte
    "preventa": "vendedor tecnico",
    "pre-venta": "vendedor tecnico",
    "tecnicomercial": "vendedor tecnico",
    
    "electromedicina": "tecnico electromedicina",
    "electromédico": "tecnico electromedicina", 
    "electromedico": "tecnico electromedicina",
    
    "imagenes diagnosticas": "tecnico imagenes medicas",
    "imágenes diagnósticas": "tecnico imagenes medicas",
    "diagnosticas": "tecnico imagenes medicas",
    "diagnósticas": "tecnico imagenes medicas"
}


# ========================================
# 4️⃣ LISTA DE UBICACIONES (Costa Rica)
# ========================================
ubicaciones_cr = [
    # Provincias
    "costa rica", "san jose", "alajuela", "cartago", "heredia", "guanacaste", "puntarenas", "limon", "limón",
    
    # Cantones de San José
    "san jose", "escazu", "escazú", "desamparados", "puriscal", "tarrazu", "tarrazú", "aserri", "aserí", "mora", 
    "goicoechea", "santa ana", "alajuelita", "coronado", "acosta", "tibas", "tibás", "moravia", "montes de oca", 
    "turrubares", "dota", "curridabat", "perez zeledon", "perez zeledón", "leon cortes", "león cortés","pavas", "zapote"
    
    # Cantones de Alajuela
    "alajuela", "san ramon", "san ramón", "grecia", "san mateo", "atenas", "naranjo", "palmares", "poas", "poás", 
    "orotina", "san carlos", "alfaro ruiz", "valverde vega", "upala", "los chiles", "guatuso", "rio cuarto", "río cuarto",
    
    # Cantones de Cartago
    "cartago", "paraiso", "paraíso", "la union", "la unión", "jimenez", "jiménez", "turrialba", "alvarado", "oreamuno", "el guarco",
    
    # Cantones de Heredia
    "heredia", "barva", "santo domingo", "santa barbara", "santa bárbara", "san rafael", "san isidro", "belen", "belén", 
    "flores", "san pablo", "sarapiqui", "sarapiquí",
    
    # Cantones de Guanacaste
    "liberia", "nicoya", "santa cruz", "bagaces", "carrillo", "canas", "cañas", "abangares", "tilaran", "tilarán", 
    "nandayure", "la cruz", "hojancha","Playa", "Conchal", "playa", "conchal",
    
    # Cantones de Puntarenas
    "puntarenas", "esparza", "buenos aires", "montes de oro", "osa", "quepos", "golfito", "coto brus", "parrita", "corredores", "garabito","Quepos","quepos",
    "Ciudad","Quesada", "ciudad" "quesada", "Barranca","barranca",
    
    # Cantones de Limón
    "limon", "limón", "pococi", "pococí", "siquirres", "talamanca", "matina", "guacimo", "guácimo", "Orotina","orotina","Puriscal","puriscal",
    
    # Distritos y lugares específicos
    "sabana", "sabana norte", "sabana sur", "merced", "hospital", "catedral", "zapote", "san francisco", "ulloa", 
    "santa ana", "santa ana", "salitral", "piedades", "san antonio", "guadalupe", "san rafael", "san isidro", 
    "concepcion", "concepción", "san felipe", "patarra", "san gabriel", "la aurora", "sanjose", "sanjosé", "uruca", "Parrita","parrita", "Jacó","Jaco","jaco",
    "Florencia", "florencia",
    
    # Zonas y regiones
    "gam", "gran area metropolitana", "gran área metropolitana", "area metropolitana", "área metropolitana",
    "zona norte", "zona sur", "zona atlantica", "zona atlántica", "zona pacifico", "zona pacífico",
    "centro", "centro ciudad", "centro urbano", "centro comercial", "centro empresarial", "playa",
    
    # Barrios y urbanizaciones
    "rohrmoser", "lindora", "multiplaza", "escazu", "santa ana", "ciudad colon", "san pedro", "los yoses",
    "barrio escalante", "la california", "sabanilla", "moravia", "tibas", "desamparados", "alajuelita",
    "curridabat", "san francisco", "guadalupe", "coronado", "santo domingo", "santa barbara", "san rafael","purral","San Sebastian",
    
    # Zonas francas y parques industriales
    "zona franca", "ultrapark", "coyol", "america free zone", "global park", "lindora", "zurqui","zona", "GAM",
    
    # Referencias geográficas
    "cerca", "cercano", "cercana", "alrededores", "zona", "sector", "area", "área", "region", "región",
    "local", "ubicado", "ubicada", "lugar", "sitio", "punto", "centro", "central",
    
    # Distritos específicos encontrados
    "purral", "rio segundo", "río segundo", "san sebastian", "san sebastián",
    "san joaquin", "san joaquín", "san pablo", "san luis", "san rafael", 
    "san isidro", "santo domingo", "tibas", "tibás", "guadalupe", "coronado",
    "sabanilla", "curridabat", "desamparados", "alajuelita", "escazu", "escazú",
    "santa ana", "ciudad colon", "ciudad colón", "san pedro", "los yoses",
    "barrio escalante", "la california", "moravia", "goicoechea", "montes de oca",
    "san francisco", "ulloa", "rohrmoser", "lindora", "multiplaza",
    "coyol", "zona franca", "america free zone", "global park", "zurqui",
    
    # Zonas específicas mencionadas
    "gam", "gran area metropolitana", "zona norte", "zona sur", "zona atlantica",
    "zona atlántica", "zona pacifico", "zona pacífico", "centro", "centro ciudad",
    
    # Nuevos cantones y distritos
    "turrialba", "paraíso", "la union", "la unión", "el guarco", "orotina",
    "grecia", "san ramon", "san ramón", "san carlos", "pital", "florencia",
    "liberia", "nicoya", "santa cruz", "bagaces", "carrillo", "canas", "cañas",
    "puntarenas", "esparza", "buenos aires", "quepos", "golfito", "limon", "limón",
    "guacimo", "guácimo", "matina", "siquirres", "pococi", "pococí",
    
    # Barrios y urbanizaciones específicas
    "sabana", "sabana norte", "sabana sur", "merced", "hospital", "catedral",
    "zapote", "piedades", "san antonio", "san gabriel", "la aurora", "uruca",
    "san vicente", "san miguel", "san jose", "sanjose", "sanjosé"
]

# ========================================
# 5️⃣ PALABRAS DE RUIDO
# ========================================
palabras_ruido = [
     # Términos específicos de tu dataset
    "back", "up", "back up", "medio", "tiempo", "completo", "parcial",
    "diurno", "nocturno", "rotativo", "guardia", "guardias", "horario",
    "jornada", "turno", "zona", "sector", "area", "área", "alrededores",
    "centro", "sur", "norte", "este", "oeste",
    
    # Especificaciones técnicas
    "b1", "b2", "b3", "licencia", "vehiculo", "vehículo", "transporte",
    "propio", "movilizarse", "trasladarse", "giras", "viajar",
    
    # Requisitos educativos
    "bachiller", "bachillerato", "tecnico", "técnico", "tecnologo", "tecnólogo",
    "universitario", "universitaria", "estudiante", "egresado", "egresada",
    "graduado", "graduada", "diplomado", "diplomada", "certificado", "certificada",
    
    # Experiencia
    "experiencia", "experto", "experta", "principiante", "avanzado", "avanzada",
    "intermedio", "intermedia", "basico", "básico", "nivel",
    
    # Términos de proceso
    "proceso", "procesos", "sistema", "sistemas", "equipo", "equipos",
    "herramienta", "herramientas", "mantenimiento", "correctivo", "preventivo",
    "predictivo", "reparacion", "reparación", "instalacion", "instalación",
    
    # Especificaciones de equipo
    "maquinaria", "pesada", "liviana", "industrial", "comercial", "residencial",
    "electromecanico", "electromecánico", "electronico", "electrónico",
    "mecanico", "mecánico", "electrico", "eléctrico",
    
    # Términos médicos y científicos
    "medico", "médico", "clinico", "clínico", "laboratorio", "farmaceutico", "farmacéutico",
    "microbiologo", "microbiólogo", "quimico", "químico", "fisico", "físico",
    
    # Comercial y ventas
    "comercial", "ventas", "vendedor", "vendedora", "ejecutivo", "ejecutiva",
    "asesor", "asesora", "consultor", "consultora", "cliente", "clientes",
    
    # Seguridad
    "seguridad", "protector", "protectora", "vigilante", "guardia", "oficial",
    "monitoreo", "alarma", "camaras", "cámaras",
    
    # Términos generales de empleo
    "puesto", "posicion", "posición", "vacante", "empleo", "trabajo",
    "funciones", "responsabilidades", "labores", "tareas", "actividades",
    "requisitos", "indispensable", "deseable", "preferible", "indispensables",
    
    # Palabras vacías adicionales
    "de", "en", "para", "con", "y", "o", "a", "la", "el", "los", "las",
    "un", "una", "unos", "unas", "su", "sus", "se", "por", "para", "al",
    "del", "lo", "le", "les", "nos", "os", "te", "me", "mi", "tu", "tu",
    # ========== LICENCIAS Y TRANSPORTE ==========
    "licencia", "b1", "b2", "b3", "b4", "a1", "a2", "a3", "c1", "c2", "c3",
    "al dia", "al día", "vigente", "actualizada", "actualizado", "vencida", "vencido",
    "montacargas", "vehiculo", "vehículo", "automóvil", "automovil", "carro", "auto",
    "moto", "motocicleta", "bicicleta", "transporte", "transporte propio",
    "vehiculo propio", "vehículo propio", "transporte publico", "transporte público",
    "bus", "autobus", "autobús", "taxi", "uber", "did", "conductor", "chofer",
    
    # ========== DISPONIBILIDAD ==========
    "disponibilidad", "disponible", "disponibles", "inmediata", "inmediato", "inmediatas",
    "rapida", "rápida", "rapido", "rápido", "pronta", "pronto", "urgente", "urgentes",
    "viajar", "viajes", "giras", "trasladarse", "movilizarse", "movilidad", "traslado",
    "desplazarse", "desplazamiento", "itinerante", "movil", "móvil", "flexibilidad",
    "flexible", "adaptabilidad", "adaptable", "cambio", "cambios", "rotacion", "rotación","transporte","publico"
    
    # ========== EXPERIENCIA ==========
    "experiencia", "experiencias", "comprobada", "comprobado", "comprobadas", "comprobados",
    "deseable", "deseables", "preferible", "preferibles", "requerida", "requerido",
    "necesaria", "necesario", "indispensable", "indispensables", "obligatoria", "obligatorio",
    "años", "anos", "año", "ano", "años de experiencia", "anos de experiencia",
    "minimo", "mínimo", "maximo", "máximo", "al menos", "por lo menos", "mas de", "más de",
    "menos de", "entre", "desde", "hasta", "promedio", "amplia", "amplio", "extensa", "extenso",
    
    # ========== HORARIOS Y JORNADAS ==========
    "jornada", "jornadas", "turno", "turnos", "horario", "horarios", "schedule", "scheduling",
    "diurno", "diurna", "diurnos", "diurnas", "nocturno", "nocturna", "nocturnos", "nocturnas",
    "matutino", "matutina", "matutinos", "matutinas", "vespertino", "vespertina", "vespertinos", "vespertinas",
    "rotativo", "rotativa", "rotativos", "rotativas", "flexible", "flexibles", "fijo", "fija", "fijos", "fijas",
    "completo", "completa", "completos", "completas", "parcial", "parciales", "medio", "media", "medios", "medias",
    "tiempo", "tiempos", "hora", "horas", "minuto", "minutos", "segundo", "segundos",
    "laboral", "laborales", "trabajo", "trabajos", "empleo", "empleos", "ocupacion", "ocupación",
    
    # ========== ENTORNOS LABORALES ==========
    "empresa", "empresas", "compania", "compañía", "company", "corporacion", "corporación",
    "corporation", "organizacion", "organización", "organization", "institucion", "institución",
    "institution", "entidad", "entidades", "grupo", "grupos", "holding", "multinacional",
    "nacional", "internacional", "local", "locales", "global", "regional", "privada", "privado",
    "publica", "público", "gubernamental", "estatal", "independiente", "propia", "propio",
    "contrato", "contratos", "indefinido", "indefinida", "indefinidos", "indefinidas",
    "temporal", "temporales", "definido", "definida", "definidos", "definidas", "duracion", "duración",
    "periodo", "período", "proyecto", "proyectos", "obra", "obras", "servicio", "servicios",
    "corporativo", "corporativa", "corporativos", "corporativas", "comercial", "comerciales",
    "industrial", "industriales", "manufacturera", "manufacturero", "manufactureras", "manufactureros",
    "produccion", "producción", "productivo", "productiva", "productivos", "productivas",
    "operativo", "operativa", "operativos", "operativas", "planta", "plantas", "fabrica", "fábrica",
    "bodega", "bodegas", "almacen", "almacén", "deposito", "depósito", "centro", "centros",
    "oficina", "oficinas", "sucursal", "sucursales", "local", "locales", "tienda", "tiendas",
    "negocio", "negocios", "establecimiento", "establecimientos", "punto", "puntos",
    "supermercado", "supermercados", "mall", "centro comercial", "plaza", "plazas",
    
    # ========== VERBOS DE SOLICITUD ==========
    "se", "requiere", "requerimos", "solicitamos", "buscamos", "necesitamos", "precisamos",
    "buscamos", "encontramos", "encontremos", "contratamos", "incorporamos", "integramos",
    "sumamos", "unimos", "agregamos", "añadimos", "adicionamos", "incluimos", "aceptamos",
    "recibimos", "tomamos", "captamos", "atrapamos", "enganchamos", "fichamos", "alistamos",
    "preparamos", "formamos", "entrenamos", "capacitamos", "adiestramos", "instruimos",
    "educamos", "enseñamos", "mostramos", "demostramos", "presentamos", "exponemos",
    "ofrecemos", "proporcionamos", "brindamos", "damos", "entregamos", "cedemos",
    "otorgamos", "concedemos", "facilitamos", "permitimos", "autorizamos", "habilitamos",
    "activamos", "iniciamos", "comenzamos", "empezamos", "arrancamos", "partimos",
    "lanzamos", "desplegamos", "implementamos", "ejecutamos", "realizamos", "efectuamos",
    "hacemos", "elaboramos", "producimos", "generamos", "creamos", "construimos",
    "desarrollamos", "evolucionamos", "mejoramos", "optimizamos", "perfeccionamos",
    "pulimos", "afinamos", "ajustamos", "adaptamos", "adecuamos", "acomodamos",
    "modificamos", "cambiamos", "alteramos", "transformamos", "convertimos", "mutamos",
    "evolucionamos", "progresamos", "avanzamos", "proseguimos", "continuamos", "seguimos",
    "mantenemos", "conservamos", "preservamos", "protegemos", "resguardamos", "custodiamos",
    "vigilamos", "supervisamos", "controlamos", "monitoreamos", "revisamos", "verificamos",
    "comprobamos", "confirmamos", "certificamos", "avalamos", "garantizamos", "aseguramos",
    "respaldamos", "apoyamos", "sostenemos", "sustentamos", "financiamos", "fondemos",
    "patrocinamos", "auspiciamos", "promovemos", "impulsamos", "motivamos", "incentivamos",
    "estimulamos", "alentamos", "animamos", "inspiramos", "guiamos", "dirigimos",
    "conducimos", "lideramos", "coordinamos", "organizamos", "estructuramos", "sistematizamos",
    "planificamos", "planeamos", "programamos", "agendamos", "calendarizamos", "cronogramamos",
    "presupuestamos", "cotizamos", "presupuestamos", "calculamos", "estimamos", "evaluamos",
    "valoramos", "tasamos", "ponderamos", "medimos", "cuantificamos", "dimensionamos",
    "analizamos", "estudiamos", "investigamos", "examinamos", "inspeccionamos", "auditamos",
    "diagnosticamos", "detectamos", "identificamos", "reconocemos", "discriminamos",
    "diferenciamos", "distinguimos", "separamos", "clasificamos", "categorizamos",
    "tipificamos", "etiquetamos", "rotulamos", "marcamos", "señalamos", "indicamos",
    "mostramos", "señalizamos", "guiamos", "orientamos", "dirigimos", "encaminamos",
    "encauzamos", "canalizamos", "conducimos", "llevamos", "traemos", "transportamos",
    "movemos", "desplazamos", "trasladamos", "cambiamos", "modificamos", "alteramos",
    "transformamos", "convertimos", "mutamos", "evolucionamos", "progresamos", "avanzamos",
    
    # ========== EDUCACIÓN Y FORMACIÓN ==========
    "educacion", "educación", "education", "formacion", "formación", "training", "capacitacion", "capacitación",
    "estudio", "estudios", "study", "academico", "académico", "academica", "académica", "academic", "académicos", "académicas",
    "bachillerato", "bachiller", "secundaria", "colegio", "primaria", "universidad", "universitario", "universitaria",
    "licenciatura", "licenciado", "licenciada", "grado", "postgrado", "maestria", "maestría", "doctorado", "phd",
    "diplomado", "diploma", "certificado", "certificacion", "certificación", "certification", "titulo", "título", "degree",
    "estudiante", "estudiantes", "student", "graduado", "graduada", "graduate", "egresado", "egresada", "alumno", "alumna",
    "profesor", "profesora", "teacher", "instructor", "instructora", "trainer", "facilitador", "facilitadora", "tutor", "tutora",
    "curso", "cursos", "course", "taller", "talleres", "workshop", "seminario", "seminarios", "seminar", "conferencia", "conferencias",
    "conocimiento", "conocimientos", "knowledge", "saber", "saberes", "aprendizaje", "learning", "enseñanza", "teaching",
    "habilidad", "habilidades", "skill", "skills", "capacidad", "capacidades", "ability", "competencia", "competencias", "competence",
    "aptitud", "aptitudes", "aptitude", "talento", "talentos", "talent", "don", "dones", "gift", "vocacion", "vocación", "vocation",
    "experto", "experta", "expert", "especialista", "specialist", "profesional", "professional", "Staff", "staff", "carga", "liviana" , "pesada",
    "Inventarios", "Comercial","Facilidades", "Flotilla", 
    # Ubicaciones y referencias espaciales
    "sector", "zona", "ubicado en", "ubicada en", "oficina", "supermercado", 
    "bodega", "tienda", "local", "centro", "alrededores", "area", "área",
    
     # Términos específicos de tu dataset que son ruido
    "policy", "asuntos", "publicos", "proyectos", "ingles", "portuges", 
    "español", "idioma", "nivel", "conocimiento", "manejo", "dominio",
    "requisitos", "indispensable", "deseable", "preferible", "indispensables",
    "funciones", "responsabilidades", "labores", "tareas", "actividades",
    "proceso", "procesos", "sistema", "sistemas", "equipo", "herramientas",
    "conocimientos", "habilidades", "capacidades", "competencias",
    "brindar", "realizar", "ejecutar", "coordinar", "gestionar", "administrar",
    "supervisar", "velar", "garantizar", "asegurar", "cumplir", "seguimiento","montacarga", "activos", "fijos" , "costos",
    "REF#280790",""
    
    # Palabras vacías
    "a", "o", "en", "para", "de", "con", "y", "el", "la", "los", "las", 
    "urgente", "importante", "nuevo", "nueva", "iii", "i", "ii", "iv", "v",
    "medio", "completo", "parcial", "tiempo", "jornada", "turno", "al","ti", "carrera", "afin", "limun","back" "office","tc"
    "en", "buenas" ,"practicas", "centro","bilingue", "nuestro", "cliente", "coopealianza", "terceros","Sendgrid",".com", "com",".org","org","Trello",
   "Consumidor","Contacto","contacto", "Usuarios", "usuarios", "del", "MaxiPali","maxipali", "REF#251683","portugues"
] + ubicaciones_cr 

# ========================================
#  FUNCIÓN DE LIMPIEZA TOTAL OPTIMIZADA
# ========================================
import re

def limpiar_titulo_puro(titulo):
    if pd.isna(titulo):
        return ""

    s = str(titulo).lower().strip()

    # 1️⃣ Normalizar texto y quitar ruido general
    s = (s.replace("á", "a").replace("é", "e")
           .replace("í", "i").replace("ó", "o")
           .replace("ú", "u").replace("ñ", "n"))
    s = re.sub(r"\(.*?\)", " ", s)
    s = re.sub(r"[^\w\s&+]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    # 2️⃣ Eliminar licencias y códigos tipo B1, A2, etc.
    s = re.sub(r"\b[ab]\d+\b", " ", s)

    # 3️⃣ Tokenizar
    palabras = s.split()

    # 4️⃣ Clasificar cada palabra según los diccionarios
    puestos_keys = set(puesto_dict.keys())
    ubicaciones = set(ubicaciones_cr)
    ruido = set(palabras_ruido)
    seniority = set(seniority_dict.keys())

    resultado = []
    for i, p in enumerate(palabras):
        # a. Si es una ubicación o nivel → eliminar
        if p in ubicaciones or p in seniority:
            continue

        # b. Si está en diccionario de puestos → siempre conservar
        if p in puestos_keys:
            resultado.append(puesto_dict.get(p, p))
            continue

        # c. Si está en ruido → conservar solo si:
        #   - Está entre dos palabras de puesto, o
        #   - Es “de”, “en”, “para” y hay una palabra de puesto cerca
        if p in ruido:
            if (i > 0 and palabras[i-1] in puestos_keys) or (i < len(palabras)-1 and palabras[i+1] in puestos_keys):
                resultado.append(p)
            elif p in {"de", "en", "para", "y"}:
                resultado.append(p)
            else:
                continue
        else:
            resultado.append(p)

    # 5️⃣ Mapear sinónimos al final
    normalizadas = [puesto_dict.get(p, p) for p in resultado]

    # 6️⃣ Reglas de preservación de combinaciones clave
    s = " ".join(normalizadas)
    patrones_preservar = {
        r"regente\s+(de\s+)?farmaceutico": "regente farmaceutico",
        r"profesor\s+(de\s+)?ingles": "profesor ingles",
        r"oficial\s+(de\s+)?seguridad": "oficial seguridad",
        r"ingeniero.*(industrial|software|proyecto)": "ingeniero \\1",
        r"tecnico\s+(en\s+)?mantenimiento": "tecnico mantenimiento",
        r"chofer": "chofer",
    }
    for patron, reemplazo in patrones_preservar.items():
        s = re.sub(patron, reemplazo, s)

    # 7️⃣ Evitar duplicados o redundancias
    s = re.sub(r"\b(\w+)( \1\b)+", r"\1", s)
    s = re.sub(r"\s+", " ", s).strip()

    # 8️⃣ Si quedó vacío, rescatar la primera palabra del diccionario original
    if not s:
        for p in palabras:
            if p in puestos_keys:
                s = puesto_dict.get(p, p)
                break

    return s




# ========================================
# 7️⃣ APLICAR LIMPIEZA
# ========================================
df["titulo_es_clean"] = df["titulo_es"].map(limpiar_titulo_puro)

# ========================================
# 8️⃣ FRECUENCIA DE PALABRAS (para revisar)
# ========================================
todas_palabras = " ".join(df["titulo_es_clean"].dropna()).split()
freq = Counter(todas_palabras)
print("Palabras más frecuentes (posibles ruidos aún):")
for palabra, cnt in freq.most_common(30):
    print(f"{palabra:20s} -> {cnt}")


# ========================================
# 🔟 GUARDAR RESULTADO
# ========================================
# Guardar el DataFrame con la columna limpiada
df.to_csv('datos_limpios.csv', index=False, encoding='utf-8')
print(df.head())


Palabras más frecuentes (posibles ruidos aún):
de                   -> 533
tecnico              -> 247
en                   -> 153
ingeniero            -> 144
redes                -> 119
sociales             -> 117
profesor             -> 116
contenido            -> 115
&                    -> 115
y                    -> 62
vendedor             -> 61
para                 -> 48
a                    -> 43
san                  -> 40
jose                 -> 36
electromecanico      -> 33
operador             -> 31
mantenimiento        -> 30
farmaceutico         -> 26
medico               -> 26
hvac                 -> 26
analista             -> 24
developer            -> 23
seguridad            -> 23
datos                -> 23
civil                -> 21
industrial           -> 19
mecanico             -> 17
asesor               -> 17
eliminacion          -> 16
         Fuente            Categoría  \
0  Computrabajo  Software Developers   
1  Computrabajo  Software Developers   
2  Computrabaj

In [52]:
# ========================================
#  Prueba de titulos 
# ========================================
# --- Vista de prueba: comparar títulos originales vs títulos limpios (sin NaN) ---

# Filtrar filas que no sean NaN en ambas columnas
df_validos = df.dropna(subset=["titulo_es", "titulo_es_clean"])

# Tomar una muestra aleatoria de hasta 20 filas
muestra = df_validos[["titulo_es", "titulo_es_clean"]].sample(
    n=min(20, len(df_validos)),
    random_state=42
)

# Mostrar en pantalla
import pandas as pd
pd.set_option("display.max_colwidth", 100)
display(muestra)

# (Opcional) Guardar la muestra para revisar manualmente
#print("Archivo 'muestra_titulos_limpios.csv' guardado exitosamente.")


,titulo_es,titulo_es_clean
1612,Analista de Operaciones,analista de operaciones
9028,Content & Social Lead,contenido & redes sociales
1528,Pasante de IT,de it
1532,Pasantía Académica / Research and Development (Desarrollo web),pasantia academica investigador and development
1545,Ingeniero(a) Automatización de Pruebas de Software,ingeniero software
2034,Misceláneo(a),miscelaneo
2615,Medical Scientific Liaison - MSL (Cáncer de Mama),medical scientific liaison msl
2380,Chofer B3 | Escazú,
2284,Cocinera,chef
40,Técnico en Mantenimiento Preventivo,tecnico en tecnico mantenimiento preventivo
